# Mobile sensing Lab 2 - Python framework for data processing - Part 2 (Data science)
We will develop a machine learning pipeline for activity recognition from smartphone sensor data (acceleration, gravity, etc.). The first part of the pipeline contains:

1. Exploratory data analysis - check data quality and calculate informative statistics about the dataset;
2. Segmentation and filtering - segment the data into windows (e.g., 4 seconds data) and try different filtering methods to improve the sensor data quality i.e., remove noise;

The second part of the pipeline contains:
3. Feature extraction and visualization - calculate features for each segment, which will be used as input to machine learning models;
4. Building and evaluation of machine learning models.

## 1. Import data

The dataset that we are going to use is downloaded form: https://github.com/mmalekzadeh/motion-sense.

The dataset includes raw data from 4 sensors, 3-axis per sensor, thus 4x3 = 12 time-series. More specifically, the sensors are:
1. Orientation sensor - measures degrees of rotation that a device makes around all three physical axes x, y, z (roll, pitch, yaw);
2. Gravity sensor - measures the force of gravity in m/s2 that is applied to a device on all three physical axes x, y and z; 
3. Accelerometer - measures the acceleration force in m/s2 that is applied to a device on all three physical axes x, y, and z, including the force of gravity;
4. Gyroscope sensors - measures a device's rate of rotation in rad/s around each of the three physical axes x, y, and z. 
    
  
The dataset is collected with an iPhone 6s kept in the participant's front pocket. All data is collected in 50Hz sample rate. A total of 24 participants in a range of gender, age, weight, and height performed 6 activities in 15 trials in the same environment and conditions: downstairs, upstairs, walking, jogging, sitting, and standing. 

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
import seaborn as sns

In [ ]:
#import the data
data_path = os.getcwd()+"/motion-sense-processed.csv"
print("Loading data from:",data_path)
data = pd.read_csv(data_path,header=0,index_col =0)
print("Done. Loaded data frame (rows, columns):",data.shape, "")

Loading data from: /Users/martingjoreski/Desktop/MS/Lab2/motion-sense-processed.csv
Done. Loaded data frame (rows, columns): (1412865, 19) 


/Users/martingjoreski/anaconda3/envs/jupyter/lib/python3.5/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## 3. Feature extraction and visualization

1. Create a function "segment_filter_extract" for signal segmentation, filtering and feature extraction.
2. Extract features and check for NULL values
3. Visualize the distribution of each feature using hisotgrams
4. Visualize the distribution of each feature using boxplots
5. Normalize the features and repeat the visualization

In [ ]:
def low_pass_filter(segment):
    fs = 50  # Sampling frequency
    fc = 5  # Cut-off frequency of the filter
    w = fc / (fs / 2) # Normalize the frequency
    n= 5 #The order of the filter.
    b, a = signal.butter(n, w, 'low')
    output = signal.filtfilt(b, a, segment)
    return output

sensor_columns = sensor_columns = data.columns[:12]
stat_feature_names = ['window_counter','moment1','moment2','moment3','moment4',
                      'derivative1','derivative2',
                      'quartile25','quartile75','quartileDeviaton',
                      'min_max','variation_coeff']

#the function takes as input a segment
#and calcualtes a list of statistical features
#the function returns a list of the extracted features

def get_statistical_features(arr):
    arr =  pd.Series(arr)
    r = [arr.mean(),arr.std(),arr.skew(),arr.kurtosis(),arr.diff().mean(),
         arr.diff().diff().mean(),arr.quantile(0.25),arr.quantile(0.75),
         arr.quantile(0.75)-arr.quantile(0.25),arr.max()-arr.min()]
    if(arr.std()!=0): #avoid 0 division
        r.append(arr.mean()/arr.std())
    else:
         r.append(0.0)
    return np.hstack(r)

#the function takes as input a 1-D sifnal of a trial (pandas timeseries) 
#and the size of the window (default 4 seconds)to be used for the segmentation
#the function filters the signal, extracts features 
#and returns a list of segments represented via the features, 
#where the first "feature" is the segment's order number in the trial
def segment_filter_extract(trial, window_size=4*50):
    trial_f  = low_pass_filter(trial.values) 
    #segment the data
    win_start = 0
    win_end = win_start+window_size
    features_arr = []
    segment_counter = 0
    while win_end<len(trial_f):
        segment = trial_f[win_start:win_end]
        segment_features = get_statistical_features(segment)
        segment_features = np.concatenate(([segment_counter],segment_features))
        features_arr.append(segment_features)
        #move/slide the window
        win_start = win_end
        win_end = win_start+window_size
        segment_counter = segment_counter+1
    return features_arr

#iterate through the data of each participant and each trial 
#and perform segmentation, filtering and feature extraction
all_features = []
for participant in data.id.unique():
    data_p = data[data.id==participant] #get the data of the specific participant
    print("Segmenting data for participant:", participant)
    print("Data shape:", data_p.shape)
    participant_features = []
    for trial in data_p.trial.unique():
        data_p_t = data_p[data_p.trial==trial] #get the data of the specific participant's trial
        trial_label = data_p_t.act.median()
        print("Trial data shape:", data_p_t.shape)
        trial_features = []
        for sensor_axis in sensor_columns: #for each sensor axis perform: segmentation, filtering and feature extraction
            print(participant,trial,"Segmenting sensor-axis:", sensor_axis)
            axis_features = segment_filter_extract(data_p_t[sensor_axis]) 
            print("Number of segments,features:", np.array(axis_features).shape)
            trial_features.append(axis_features)
        trial_features = np.concatenate(trial_features,axis=1) #rows represent segments, columns represent features
        print('trial_features shape',trial_features.shape)
        trial_info = [[participant,trial,trial_label]]*len(trial_features) #for each segment crate info columns
        trial_features = np.column_stack((trial_info,trial_features)) #add info columns
        participant_features.append(trial_features)
        print('-')
    participant_features = np.concatenate(participant_features)
    print('participant_features shape',participant_features.shape)
    all_features.append(participant_features)
    print('---------')

print('DONE')
all_features= np.concatenate(all_features)
print(all_features.shape)

Segmenting data for participant: 0.0
Data shape: (62312, 19)
Trial data shape: (1751, 19)
0.0 1.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (8, 12)
0.0 1.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (8, 

Number of segments,features: (21, 12)
0.0 8.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (21, 12)
0.0 8.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (21, 12)
0.0 8.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (21, 12)
trial_features shape (21, 144)
-
Trial data shape: (1333, 19)
0.0 15.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (6, 12)
0.0 15.0 Segmenting sensor-axis: rotationRate.y
Number o

1.0 1.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (10, 12)
1.0 1.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (10, 12)
trial_features shape (10, 144)
-
Trial data shape: (2639, 19)
1.0 2.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (13, 12)
1.0 2.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (13, 12)
1.0 2.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (13, 12)
1.0 

Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (31, 12)
1.0 15.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (31, 12)
trial_features shape (31, 144)
-
Trial data shape: (4966, 19)
1.0 9.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (24, 12)
1.0 9.0 Segmenting sensor-axis: attitude.pitc

2.0 1.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (11, 12)
2.0 1.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (11, 12)
2.0 1.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (11, 12)
trial_features shape (11, 144)
-
Trial data shape: (2911, 19)
2.0 2.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (14, 12)
2.0 2.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (14, 12)
2.0 2.0 S

...

15.0 2.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (14, 12)
15.0 2.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (14, 12)
trial_features shape (14, 144)
-
Trial data shape: (1207, 19)
15.0 11.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (6, 12)
15.0 11.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (6, 

Number of segments,features: (13, 12)
trial_features shape (13, 144)
-
Trial data shape: (4814, 19)
15.0 9.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (24, 12)
15.0 9.0 Segmenting sensor-axis: userAcceleration.y
Nu

Number of segments,features: (14, 12)
16.0 2.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (14, 12)
16.0 2.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (14, 12)
16.0 2.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (14, 12)
trial_features shape (14, 144)
-
Trial data shape: (1180, 19)
16.0 11.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: attitude.yaw
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (5, 12)
16.0 11.0 Segmenting sensor-axis: rotationRate

Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (16, 12)
16.0 9.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (16, 12)
trial_features shape (16, 144)
-
Trial data shape: (960, 19)
16.0 16.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (4, 12)
16.0 16.0 Segmenting sensor-axis: attitude.pi

17.0 11.0 Segmenting sensor-axis: gravity.x
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: gravity.y
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: gravity.z
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: rotationRate.x
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: rotationRate.y
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: rotationRate.z
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: userAcceleration.x
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: userAcceleration.y
Number of segments,features: (5, 12)
17.0 11.0 Segmenting sensor-axis: userAcceleration.z
Number of segments,features: (5, 12)
trial_features shape (5, 144)
-
Trial data shape: (3184, 19)
17.0 3.0 Segmenting sensor-axis: attitude.roll
Number of segments,features: (15, 12)
17.0 3.0 Segmenting sensor-axis: attitude.pitch
Number of segments,features: (15, 1

In [ ]:
#create pandas dataframe with the fatures
column_names = ['person','trial','label']
for sensor_axis in sensor_columns:
    for feature_name in stat_feature_names:
        column_names.append(sensor_axis+"_"+feature_name)
df_features = pd.DataFrame(all_features,columns =column_names )
df_features

In [ ]:
# save features to disk to avoid repetitive feature calculation
df_features.to_csv("motion-sense-processed-features.csv")

In [ ]:
# read features from disk
df_features = pd.read_csv("motion-sense-processed-features.csv",header=0,index_col =0)

In [ ]:
#the column window_counter repeats several times. We will keep only the first occurrence
#find columns to be removed
remove_columns = []
for c in df_features.columns:
    if "window_counter" in c and "attitude.roll_window_counter" not in c:
        remove_columns.append(c)
#remove columns      
df_features = df_features.drop(remove_columns,axis=1)

In [ ]:
#The dataframe df_features contains 6886 instances (segments) and 136 columns. 
#The first 4 columns are the info columns (participant id, trial id, label and window number)
#The rest of the columns are feature that can be used as input to typical machine learning algorithms
df_features.shape

In [ ]:
#Check for NULL values
df_features.isnull().sum() #counts the number of null values for each feature. In this case, it shoud be 0

In [ ]:
#For each sensor visualize the distribution of each feature using histograms 
for sensor in ['attitude', 'gravity', 'rotationRate','userAcceleration']:
    sensor_features = [] #find the feature for the specific sensor
    for c in df_features.columns:
        if sensor in c and 'window_count' not in c:
            sensor_features.append(c)
    df_features[sensor_features].hist(figsize=(15,10))
    plt.tight_layout()
    plt.show()

In [ ]:
#For each sensor visualize the distribution of each feature using boxplots 
for sensor in ['attitude', 'gravity', 'rotationRate','userAcceleration']:
    sensor_features = [] #find the feature for the specific sensor
    for c in df_features.columns:
        if sensor in c and 'window_count' not in c:
            sensor_features.append(c)
    df_features[sensor_features].boxplot(figsize=(15,5))
    plt.tight_layout()
    plt.show()

In [ ]:
#Normalize the features (min-max normalization)
feature_columns = df_features.columns[4:]
df_features_norm = df_features.copy()
df_features_norm[feature_columns] = (df_features_norm[feature_columns]  - df_features_norm[feature_columns].min()) / (df_features_norm[feature_columns].max()-df_features_norm[feature_columns].min()) 

In [ ]:
#For each sensor visualize the distribution of each feature using histograms 
for sensor in ['attitude', 'gravity', 'rotationRate','userAcceleration']:
    sensor_features = [] #find the feature for the specific sensor
    for c in df_features.columns:
        if sensor in c and 'window_count' not in c:
            sensor_features.append(c)
    df_features_norm[sensor_features].hist(figsize=(15,10))
    plt.tight_layout()
    plt.show()

### 3.1. TO DOs
1. How would you parallelize the feature-extraction process?
2. How did the normalization influence the histograms? What about the boxplots?
3. The function "get_statistical_features" calculates only statistical (time-domain features) features. How would you calculate frequency-domain features?

In [ ]:
#For each sensor visualize the distribution of each feature using boxplots 
for sensor in ['attitude', 'gravity', 'rotationRate','userAcceleration']:
    sensor_features = [] #find the feature for the specific sensor
    for c in df_features.columns:
        if sensor in c and 'window_count' not in c:
            sensor_features.append(c)
    df_features_norm[sensor_features].boxplot(figsize=(15,5))
    plt.tight_layout()
    plt.show()

## 4. Machine learning evaluation and model building

- split the data (features) on training and test subsets using a random 70%-30% split.
- split the data (features) on training and test subsets using an ordered 70%-30% split.
- build a simple decision tree model for activity recognition and comapre its results using the two splits
- visualize the model

In [ ]:
#helper functions for calculating machine learning evaluation scores
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
def full_report(Y,pred_Y):
    print('Confusion matrix')
    print('Rows - true label')
    print('Columns - predicted label')

    labels = ['dws', 'ups', 'wlk', 'jog', 'std', 'sit']
    conf_mat = confusion_matrix(Y,pred_Y)
    cr = classification_report(Y,pred_Y)
    acc = accuracy_score(Y,pred_Y)
    print(labels)
    print(conf_mat)
    print()
    print(cr)
    print("Accuracy:",acc)
    
    return

In [ ]:
#evaluation using random split
from sklearn.model_selection import train_test_split
from sklearn import tree
X_train, X_test, y_train, y_test = train_test_split(df_features_norm[feature_columns], 
                                                    df_features_norm.label, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                   shuffle=True)
clf = tree.DecisionTreeClassifier(max_depth=5)#initialize the algorithm
clf.fit(X_train,y_train)#train the model
predictions = clf.predict(X_test) #predict the test data
full_report(y_test,predictions) # show evaluaton results for the test data

In [ ]:
#evaluation using ordered split
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(df_features_norm[feature_columns], 
                                                    df_features_norm.label, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                   shuffle=False)

clf = tree.DecisionTreeClassifier(max_depth=5)#initialize the algorithm
clf.fit(X_train,y_train)#train the model
predictions = clf.predict(X_test) #predict the test data
full_report(y_test,predictions) # show evaluaton results for the test data

In [ ]:
#model visualization
plt.figure(figsize=(15,15))
tree.plot_tree(clf)  
plt.show()

In [ ]:
#if then else representataion based on https://stackoverflow.com/questions/20224526/how-to-extract-the-decision-rules-from-scikit-learn-decision-tree
from sklearn.tree import _tree
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
#     print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)
    
tree_to_code(clf,feature_columns)

### 4.1 TO DOs
- Which classes are frequently mixed by the classifier?
- How would you reimplement the decision tree model on an android device?

### Further reading

- Normalization: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html
- General feature extractor for time-series (in most cases, expert defiend features are better. Always check related work for your specific machine learning problem): https://tsfresh.readthedocs.io/en/latest/index.html
- Machine learning (scikit-learn): https://scikit-learn.org/stable/tutorial/basic/tutorial.html
-  More info about the dataset and analysis performed over it:
http://doi.acm.org/10.1145/3302505.3310068 (Malekzadeh et al. "Mobile sensor data anonymization", Proceedings of the International Conference on Internet of Things Design and Implementation, 2019).